
# Decoding a video with SimpleVideoDecoder

In this example, we'll learn how to decode a video using the
:class:`~torchcodec.decoders.SimpleVideoDecoder` class.


First, a bit of boilerplate: we'll download a video from the web, and define a
plotting utility. You can ignore that part and jump right below to
`creating_decoder`.



In [ ]:
from typing import Optional
import torch
import requests


# Video source: https://www.pexels.com/video/dog-eating-854132/
# License: CC0. Author: Coverr.
url = "https://videos.pexels.com/video-files/854132/854132-sd_640_360_25fps.mp4"
response = requests.get(url)
if response.status_code != 200:
    raise RuntimeError(f"Failed to download video. {response.status_code = }.")

raw_video_bytes = response.content


def plot(frames: torch.Tensor, title : Optional[str] = None):
    try:
        from torchvision.utils import make_grid
        from torchvision.transforms.v2.functional import to_pil_image
        import matplotlib.pyplot as plt
    except ImportError:
        print("Cannot plot, please run `pip install torchvision matplotlib`")
        return

    plt.rcParams["savefig.bbox"] = 'tight'
    fig, ax = plt.subplots()
    ax.imshow(to_pil_image(make_grid(frames)))
    ax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])
    if title is not None:
        ax.set_title(title)
    plt.tight_layout()


## Creating a decoder

We can now create a decoder from the raw (encoded) video bytes. You can of
course use a local video file and pass the path as input, rather than download
a video.



In [ ]:
from torchcodec.decoders import SimpleVideoDecoder

# You can also pass a path to a local file!
decoder = SimpleVideoDecoder(raw_video_bytes)

The has not yet been decoded by the decoder, but we already have access to
some metadata via the ``metadata`` attribute which is a
:class:`~torchcodec.decoders.VideoStreamMetadata` object.



In [ ]:
print(decoder.metadata)

## Decoding frames by indexing the decoder



In [ ]:
first_frame = decoder[0]  # using a single int index
every_twenty_frame = decoder[0 : -1 : 20]  # using slices

print(f"{first_frame.shape = }")
print(f"{first_frame.dtype = }")
print(f"{every_twenty_frame.shape = }")
print(f"{every_twenty_frame.dtype = }")

Indexing the decoder returns the frames as :class:`torch.Tensor` objects.
By default, the shape of the frames is ``(N, C, H, W)`` where N is the batch
size C the number of channels, H is the height, and W is the width of the
frames.  The batch dimension N is only present when we're decoding more than
one frame. The dimension order can be changed to ``N, H, W, C`` using the
``dimension_order`` parameter of
:class:`~torchcodec.decoders.SimpleVideoDecoder`. Frames are always of
``torch.uint8`` dtype.




In [ ]:
plot(first_frame, "First frame")

In [ ]:
plot(every_twenty_frame, "Every 20 frame")

## Iterating over frames

The decoder is a normal iterable object and can be iterated over like so:



In [ ]:
for frame in decoder:
    assert (
        isinstance(frame, torch.Tensor)
        and frame.shape == (3, decoder.metadata.height, decoder.metadata.width)
    )

## Retrieving pts and duration of frames

Indexing the decoder returns pure :class:`torch.Tensor` objects. Sometimes, it
can be useful to retrieve additional information about the frames, such as
their :term:`pts` (Presentation Time Stamp), and their duration.
This can be achieved using the
:meth:`~torchcodec.decoders.SimpleVideoDecoder.get_frame_at` and
:meth:`~torchcodec.decoders.SimpleVideoDecoder.get_frames_at`  methods, which
will return a :class:`~torchcodec.decoders.Frame` and
:class:`~torchcodec.decoders.FrameBatch` objects respectively.



In [ ]:
last_frame = decoder.get_frame_at(len(decoder) - 1)
print(f"{type(last_frame) = }")
print(last_frame)

In [ ]:
middle_frames = decoder.get_frames_at(start=10, stop=20, step=2)
print(f"{type(middle_frames) = }")
print(middle_frames)

In [ ]:
plot(last_frame.data, "Last frame")
plot(middle_frames.data, "Middle frames")

Both :class:`~torchcodec.decoders.Frame` and
:class:`~torchcodec.decoders.FrameBatch` have a ``data`` field, which contains
the decoded tensor data. They also have the ``pts_seconds`` and
``duration_seconds`` fields which are single ints for
:class:`~torchcodec.decoders.Frame`, and 1-D :class:`torch.Tensor` for
:class:`~torchcodec.decoders.FrameBatch` (one value per frame in the batch).



## Using time-based indexing

So far, we have retrieved frames based on their index. We can also retrieve
frames based on *when* they are displayed with
:meth:`~torchcodec.decoders.SimpleVideoDecoder.get_frame_displayed_at`, which
also returns :class:`~torchcodec.decoders.Frame`.



In [ ]:
frame_at_2_seconds = decoder.get_frame_displayed_at(seconds=2)
print(f"{type(frame_at_2_seconds) = }")
print(frame_at_2_seconds)
plot(frame_at_2_seconds.data, "Frame displayed at 2 seconds")